In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from lsst.daf.butler import Butler
from lsst.ip.isr import IsrTask, IsrTaskConfig
from lsst.summit.utils.plotting import plot
import lsst.afw.cameraGeom.utils as camGeomUtils
import lsst.afw.math as afwMath
from lsst.afw import image
from lsst.geom import SpherePoint,Angle,Extent2I,Box2I,Extent2D,Point2D, Point2I
import lsst.afw.display as afwDisplay
from lsst.obs.lsst.cameraTransforms import LsstCameraTransforms
from lsst.obs.lsst import LsstCam

from skimage import io
from skimage.feature import hessian_matrix, hessian_matrix_eigvals
from skimage.color import rgb2gray
from skimage.filters import threshold_otsu

import cv2


In [ ]:
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation'])
camera = LsstCam.getCamera()
instrument = "LSSTCam"

In [ ]:
#expId = 2025090500272
#detName = 'R24_S11'
expId = 2025090900313
#detName = 'R41_S22'
#det = camera[detName]
#detName = 'R23_S21'
detName = 'R23_S10'
det = camera[detName]
detNum = det.getId()
print(detNum)
calexp = butler.get('preliminary_visit_image', detector=detNum, visit=expId, instrument=instrument)

In [ ]:
%matplotlib inline
x = plot(calexp, stretch='ccs')
axs = x.get_axes()
#axs[0].axis('off')
#axs[0].set_title(f"{expId} {detNum}")
x.savefig(f"/home/c/cslage/u/Satellites/images/LSSTCam_{expId}_{detNum}.png")
x

In [ ]:
x = plot(calexp.image.array[2500:3500, 1000:2000], stretch='ccs')
x

In [ ]:
plt.plot(calexp.image.array[3100, 1000:2000])

In [ ]:
ystart = 3200
xstart = 1500
w = 400
z = 400
stripe = np.zeros([w])
for i in range(z):
    x = xstart + i
    y = ystart - i
    for j in range(w):
        stripe[j] += calexp.image.array[y+j, x+j]
stripe /= z
plt.plot(stripe)

In [ ]:
x = plot(calexp.image.array[2900:3300, 1200:1600], stretch='ccs')
x

In [ ]:
arr = calexp.image.array
arr = np.clip(arr, a_min=0, a_max=100)

new_shape = (1001, 1024)

# Rebin by averaging
bin_arr = arr.reshape(
    new_shape[0],
    arr.shape[0] // new_shape[0],
    new_shape[1],
    arr.shape[1] // new_shape[1]
).mean(-1).mean(1)

In [ ]:
def find_faint_ridges(calexp, sigma=1.0, edge=200, make_plots=False):
    arr = calexp.image.array
    
    # Bin 4x4
    arr = np.clip(arr, a_min=0, a_max=100)
    #print(arr.shape)
    new_shape = (int(arr.shape[0]/4), int(arr.shape[1]/4))
    
    # Rebin by averaging
    bin_arr = arr.reshape(
        new_shape[0],
        arr.shape[0] // new_shape[0],
        new_shape[1],
        arr.shape[1] // new_shape[1]
    ).mean(-1).mean(1)
    #print(bin_arr.shape)
    
    blurred = cv2.medianBlur(bin_arr, 3)        # smooth small features
    
    # Step 2: Enhance faint streaks
    gauss = cv2.GaussianBlur(blurred, (11,11), 0)
    H_elems = hessian_matrix(gauss, sigma=sigma, order='rc')
    maxima_ridges, minima_ridges = hessian_matrix_eigvals(H_elems)
    #print(H_elems)
    #try:
    threshold = threshold_otsu(minima_ridges)# / 4.0
    print(threshold, threshold_otsu(minima_ridges))
    binary_ridges = minima_ridges < threshold
    binary_ridges = binary_ridges.astype(np.uint8)
    binary_ridges[:,0:edge] = 0
    binary_ridges[:,-edge:-1] = 0
    binary_ridges[0:edge,:] = 0
    binary_ridges[-edge:-1,:] = 0
    binary_ridges.astype(np.uint8)
    _, binary = cv2.threshold(binary_ridges, 0.5, 255, cv2.THRESH_BINARY)
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary, connectivity=8)
    # Find the largest non-background component (ignore label 0 = background)
    long_labels = []
    for i in range(num_labels):
        mask = np.uint8(labels == i)
        # Extract points (x,y) of this component
        ys, xs = np.where(mask > 0)
        points = np.column_stack((xs, ys))
        rect = cv2.minAreaRect(points)
        (center, (width, height), angle) = rect
        #print(width, height)
        if height > 0:
            aspect_ratio = max(width, height) / min(width, height)
        else:
            aspect_ratio = 0  # Handle division by zero for flat regions
        #print(i, aspect_ratio)
        if aspect_ratio > 10:
            long_labels.append(i)
    #areas = stats[1:, cv2.CC_STAT_AREA]
    #largest_label = 1 + np.argmax(areas)

    color_img = cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)
    disp_img = np.zeros_like(bin_arr)
    rows, cols = binary.shape
    # Mask only the longest ones
    #print(len(long_labels))
    for label in long_labels:
        mask = np.uint8(labels == label)
        
        # Extract points (x,y) of this component
        ys, xs = np.where(mask > 0)
        points = np.column_stack((xs, ys))
        # Fit a line through the points
        [vx, vy, x0, y0] = cv2.fitLine(points, cv2.DIST_L2, 0, 0.01, 0.01)
        # Weed out near horizontal or vertical lines
        #print(vx, vy)
        if (abs(vx) < 0.1) or (abs(vy) < 0.1):
            continue
        # Define two endpoints for drawing
        lefty = int((-x0 * vy / vx) + y0)
        righty = int(((cols - x0) * vy / vx) + y0)
        # Show result

        cv2.line(disp_img, (cols-1, righty), \
                 (0, lefty), (255,255,255), 20)
    disp_img = cv2.resize(disp_img, (arr.shape[1], arr.shape[0]), interpolation=cv2.INTER_LINEAR)
    #print(disp_img.shape)
    if make_plots:
        fig, axes = plt.subplots(2, 2, figsize=(10, 8))
        plt.suptitle(f"Streak finding {expId}_{detNum}")
        ax = axes.ravel()
        ax[0].imshow(arr, cmap=plt.cm.gray, vmin=0, vmax=100, origin='lower')
        ax[0].set_title('Original Image')
        ax[1].imshow(disp_img, cmap=plt.cm.gray, origin='lower')
        #ax[1].imshow(color_img, origin='lower')
        ax[1].set_title(f'Detected Ridges (sigma={sigma})')
        ax[2].set_title(f'Maxima Ridges (sigma={sigma})')
        ax[2].imshow(maxima_ridges, origin='lower', vmin=-.01, vmax=.01)
        ax[3].set_title(f'Minima Ridges (sigma={sigma})')
        ax[3].imshow(minima_ridges, origin='lower', vmin=-.01, vmax=.01)
    return [disp_img]#, minima_ridges]


In [ ]:
# Example usage:
[disp_img, minima_ridges] = find_faint_ridges(calexp, sigma=12.0, make_plots=True)
plt.savefig(f"/home/c/cslage/u/Satellites/streak_images/Test_{dayObs}_{seqNum}.png")

In [ ]:
plt.plot(minima_ridges[500,:])

In [ ]:
disp_img.shape

In [ ]:
plt.imshow(disp_img)

In [ ]:
expId = 2025090900313
images = {}
for detNum in range(189):
    try:
        calexp = butler.get('preliminary_visit_image', detector=detNum, visit=expId, instrument=instrument)
        [color_img] = find_faint_ridges(calexp, sigma=12.0)
        images[detNum] = color_img
        print(f"Finished detector {detNum}")
    except:
        continue

In [ ]:
images[106][:,:,0]

In [ ]:
plt.imshow(images[106][:,:,2])

In [ ]:
image.ImageF?

In [ ]:
image.ImageF?

In [ ]:
from lsst.geom import SpherePoint,Angle,Extent2I,Box2I,Extent2D,Point2D, Point2I
#test = image.ImageF(array=images[106].astype(np.float32), deep=False, xy0=Point2I(0, 0))
test = image.ImageF(array=disp_img, deep=False, xy0=Point2I(0, 0))

In [ ]:
plt.imshow(test.array)

In [ ]:
calexp.image.array.dtype

In [ ]:
rafts = [      'R01','R02','R03', 
         'R10','R11','R12','R13','R14',
         'R20','R21','R22','R23','R24',
         'R30','R31','R32','R33','R34',  
               'R41','R42','R43']
ccds = ['S00','S01','S02',
        'S10','S11','S12',
        'S20','S21','S22']
corners = ['R01_S00', 'R01_S01', 'R03_S01', 'R03_S02', \
                     'R10_S00', 'R10_S10', 'R30_S20', 'R30_S10', \
                    'R41_S20', 'R41_S21', 'R43_S21', 'R43_S22', \
                    'R34_S22', 'R34_S12', 'R14_S02', 'R14_S12']
detectorNameList = []
for raft in rafts:
    for ccd in ccds:
        name = raft+'_'+ccd
        if name not in corners:
            detectorNameList.append(name)


In [ ]:
rafts = [ 'R23']
ccds = ['S00','S01','S02',
        'S10','S11','S12',
        'S20','S21','S22']
detectorNameList = []
for raft in rafts:
    for ccd in ccds:
        detectorNameList.append(raft+'_'+ccd)
detectorNameList


In [ ]:
detectorNameList = ['R23_S21']

In [ ]:
disp_img.dtype

In [ ]:
dayObs = 20250909
seqNum = 313
detector=167
calexp = butler.get('preliminary_visit_image', detector=detector, day_obs=dayObs, seq_num=seqNum)
[disp_img] = find_faint_ridges(calexp, sigma=12.0)
oim = image.ImageF(array=disp_img, deep=False, xy0=Point2I(0, 0))
plt.imshow(oim.array, origin='lower')

In [ ]:
def streakCallback(im, ccd, imageSource):
    """Assemble the streakImages"""
    calexp = butler.get('preliminary_visit_image', detector=ccd.getId(), day_obs=dayObs, seq_num=seqNum)
    [disp_img] = find_faint_ridges(calexp, sigma=12.0)
    print(ccd.getId(), np.max(disp_img.flatten()))
    oim = image.ImageF(array=disp_img, deep=False, xy0=Point2I(0, 0))
    #print(oim.array.shape)
    return oim

def calexpCallback(im, ccd, imageSource):
    dayObs = imageSource.kwargs['day_obs']
    seqNum = imageSource.kwargs['seq_num']
    exp = imageSource.butler.get('preliminary_visit_image', detector=ccd.getId(), day_obs=dayObs, seq_num=seqNum)
    nQuarter = ccd.getOrientation().getNQuarter()
    #oim = afwMath.rotateImageBy90(exp.image.array, nQuarter)
    print(ccd.getId(), np.nanmedian(exp.image.array.flatten()))
    oim = exp.image
    print(oim.array.shape)
    return oim


def rawCallback(im, ccd, imageSource):
    """Assemble the CCD image.  Just bias subtraction and gain correction"""
    print(ccd.getId(), np.nanmedian(exp.image.array.flatten()))
    oim = camGeomUtils.rawCallback(im, ccd, imageSource,
                                   subtractBias=True, correctGain=False)
    return oim

In [ ]:
%matplotlib inline
instrument = "LSSTCam"
camera = butler.get('camera', instrument=instrument)
fig = plt.figure(figsize=(12,12))
disp = afwDisplay.Display(1, "matplotlib")
#disp.scale('linear', min=20, max=200)
#disp.setImageColormap("gray_r")
dayObs = 20250909
seqNum = 313
detectorNameList = ['R23_S10']
dataType='raw'
mos = camGeomUtils.showCamera(camera,
                              camGeomUtils.ButlerImage(butler, dataType, 
                                                       instrument=instrument, 
                                                       day_obs=dayObs, seq_num=seqNum,
                                                       verbose=False, callback=streakCallback,
                                                       background=np.nan),
                              detectorNameList=detectorNameList,
                              binSize=16, display=disp, overlay=False,
                              title="%d %d" % (dayObs, seqNum))
plt.savefig(f"/home/c/cslage/u/Satellites/streak_images/Test_{dayObs}_{seqNum}.png")

In [ ]:
%matplotlib inline
instrument = "LSSTCam"
camera = butler.get('camera', instrument=instrument)
fig = plt.figure(figsize=(12,12))
disp = afwDisplay.Display(1, "matplotlib")
disp.scale('linear', min=20, max=200)
disp.setImageColormap("gray")
dayObs = 20250909
seqNum = 313
#detectorNameList = ['R43_S11']
dataType='raw'
mos = camGeomUtils.showCamera(camera,
                              camGeomUtils.ButlerImage(butler, dataType, 
                                                       instrument=instrument, 
                                                       day_obs=dayObs, seq_num=seqNum,
                                                       verbose=False, callback=streakCallback,
                                                       background=np.nan),
                              detectorNameList=detectorNameList,
                              binSize=16, display=disp, overlay=False,
                              title="%d %d" % (dayObs, seqNum))

#plt.savefig(f"/home/c/cslage/u/LSSTCam/images/LSSTCam_Calexp_CutDown_{dayObs}_{seqNum}.png")
plt.savefig(f"/home/c/cslage/u/Satellites/streak_images/Streak_Finding_{dayObs}_{seqNum}.png")

In [ ]:
    areas = stats[1:, cv2.CC_STAT_AREA]
    largest_label = 1 + np.argmax(areas)
    
    # Mask only the largest component
    mask = np.uint8(labels == largest_label)
    
    # Extract points (x,y) of this component
    ys, xs = np.where(mask > 0)
    points = np.column_stack((xs, ys))
    
    # Fit a line through the points
    [vx, vy, x0, y0] = cv2.fitLine(points, cv2.DIST_L2, 0, 0.01, 0.01)
    print(vx, vy, x0, y0)
    # Define two endpoints for drawing
    
    lefty = int((-x0 * vy / vx) + y0)
    righty = int(((cols - x0) * vy / vx) + y0)
    
    # Show result
    color_img = cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)
    cv2.line(color_img, (cols-1, righty), (0, lefty), (0, 255, 0), 10)


In [ ]:
binary_ridges[500, 600:800]

In [ ]:
_, binary = cv2.threshold(binary_ridges, 0.5, 255, cv2.THRESH_BINARY)
binary[500, 600:800]

In [ ]:
#plt.imshow(bin_arr, origin='lower')
from skimage import morphology, segmentation, io 
result = morphology.remove_small_objects(binary_ridges, 100000)
plt.imshow(result, origin='lower')

In [ ]:
plt.plot(minima_ridges[400,:])

In [ ]:
test5 = binary_ridges.astype(np.uint8)

In [ ]:
cv2.imwrite('test5.png', test5)
#test = cv2.imread('test.png')

In [ ]:
cv2.threshold?

In [ ]:
np.max(binary_ridges)

In [ ]:
plt.imshow(binary_ridges)

In [ ]:
# Load the binary image
img = cv2.imread("binary_ridges.png", cv2.IMREAD_GRAYSCALE)

# Ensure binary (0 and 255)
_, binary = cv2.threshold(img, 0.5, 255, cv2.THRESH_BINARY)

# Find connected components
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary, connectivity=8)
# Find the largest non-background component (ignore label 0 = background)
long_labels = []
for i in range(num_labels):
    mask = np.uint8(labels == i)
    # Extract points (x,y) of this component
    ys, xs = np.where(mask > 0)
    points = np.column_stack((xs, ys))
    rect = cv2.minAreaRect(points)
    (center, (width, height), angle) = rect
    #print(width, height)
    if height > 0:
        aspect_ratio = max(width, height) / min(width, height)
    else:
        aspect_ratio = 0  # Handle division by zero for flat regions
    if aspect_ratio > 10:
        long_labels.append(i)
        #areas = stats[1:, cv2.CC_STAT_AREA]
#largest_label = 1 + np.argmax(areas)

color_img = cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)
# Mask only the largest component
for label in long_labels:
    mask = np.uint8(labels == label)
    
    # Extract points (x,y) of this component
    ys, xs = np.where(mask > 0)
    points = np.column_stack((xs, ys))
    ymin = np.min(points[0,:])
    xmin = np.min(points[:,0])
    ymax = np.max(points[0,:])
    xmax = np.max(points[:,0])
    print(xmin, ymin, xmax, ymax)
    # Fit a line through the points
    [vx, vy, x0, y0] = cv2.fitLine(points, cv2.DIST_L2, 0, 0.01, 0.01)
    print(vx, vy, x0, y0)
    # Define two endpoints for drawing
    rows, cols = binary.shape
    lefty = int((-x0 * vy / vx) + y0)
    righty = int(((cols - x0) * vy / vx) + y0)
    print(lefty, righty)
    # Show result

    #cv2.line(color_img, (min(xmax, cols-1), max(ymin, righty)), \
    #         (max(xmin, 0), min(ymax, lefty)), (0, 0, 255), 10)
    cv2.line(color_img, (cols-1, righty), \
             (0, lefty), (0, 0, 255), 10)
#print(lefty, righty)
#print((min(xmax, cols-1), max(ymin, righty)), \
#         (max(xmin, 0), min(ymax, lefty)))

plt.imshow(color_img, origin='lower')
plt.title("Longest Streak Line")
plt.show()

# Coordinates of the line
print("Line passes through: (0, {}) to ({}, {})".format(lefty, cols-1, righty))

In [ ]:
stats

In [ ]:
long_labels = []
for i in range(num_labels):
    mask = np.uint8(labels == i)
    # Extract points (x,y) of this component
    ys, xs = np.where(mask > 0)
    points = np.column_stack((xs, ys))
    rect = cv2.minAreaRect(points)
    (center, (width, height), angle) = rect
    #print(width, height)
    if height > 0:
        aspect_ratio = max(width, height) / min(width, height)
    else:
        aspect_ratio = 0  # Handle division by zero for flat regions
    if aspect_ratio > 10:
        long_labels.append(i)

In [ ]:
long_labels

In [ ]:
def remove_small_regions_cv2(binary_image, min_area_threshold):
    # Find contours in the binary image
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    print(len(contours))

    # Create a blank image to draw the filtered regions
    filtered_image = np.zeros_like(binary_image)

    for contour in contours:
        area = cv2.contourArea(contour)
        #print(area)
        if area >= min_area_threshold:
            # Draw contours with area greater than or equal to the threshold
            cv2.drawContours(filtered_image, [contour], -1, 255, -1) # Draw white filled contours

    return filtered_image


In [ ]:

filtered_image = remove_small_regions_cv2(binary_ridges, 100000)
kernel = np.ones((15, 15), np.uint8) # A 3x3 square kernel
shrunk_image = cv2.erode(filtered_image, kernel, iterations=1)
grown_image = cv2.dilate(shrunk_image, kernel, iterations=1)
shrunk_image = cv2.erode(grown_image, kernel, iterations=1)
grown_image = cv2.dilate(shrunk_image, kernel, iterations=1)
shrunk_image = cv2.erode(grown_image, kernel, iterations=1)
grown_image = cv2.dilate(shrunk_image, kernel, iterations=1)
 

plt.imshow(grown_image, cmap='gray', origin='lower')

In [ ]:
#test = minima_ridges * 1.0E7
test = np.clip(minima_ridges, a_min=-0.01, a_max=0.01)
test2 = -(test + np.min(test))
test3 = test2 * 10000
test4 = test3.astype(np.uint8)
plt.plot(test4[400,:])
#test2[100, 150:155]
#plt.imshow(test2)

In [ ]:
cv2.HoughLines?

In [ ]:
plt.imshow(binary_ridges)

In [ ]:
lines = cv2.HoughLinesP(
    edges,
    rho=200, # Distance resolution
    theta=np.pi / 180, # Angle resolution
    threshold=20, # Minimum number of votes
    minLineLength=100, # Minimum line length
    maxLineGap=20 # Maximum gap between points
)
print(f"There were {len(lines)} lines")
# 5. Draw detected lines and display the result
if lines is not None:
    for line in lines:
        print(line)
        x1, y1, x2, y2 = line[0]
        ang = abs(np.atan2((y2-y1), (x2-x1)))
        if (ang < 0.05) or (abs(ang - np.pi/2) < 0.05) or \
                          (abs(ang - np.pi) < 0.05):
            continue
        cv2.line(minima_ridges, (x1, y1), (x2, y2), (0, 255, 0), 20) # Draw lines in green

"""
lines = cv2.HoughLines(
    edges,
    rho=10, # Distance resolution
    theta=np.pi / 180, # Angle resolution
    threshold = 2
)
print(f"There were {len(lines)} lines")
# 5. Draw detected lines and display the result
if lines is not None:
    for line in lines:
        r, th = line[0]
        if (th < 0.05) or (abs(th - np.pi/2) < 0.05) or \
                          (abs(th - np.pi) < 0.05):
            continue
        cv2.line(cropped_img, (x1, y1), (x2, y2), (0, 255, 0), 2) # Draw lines in green
"""
fig = plt.figure(figsize=(10, 10))
plt.imshow(minima_ridges, origin = 'lower')


In [ ]:
type(minima_ridges)

In [ ]:
cv2.line?

In [ ]:
cv2.Canny?

In [ ]:
edges = cv2.Canny(test4, 20, 80, apertureSize=3)
plt.imshow(edges, origin='lower')

In [ ]:
expId = 2025090900313
image_path = f"/home/c/cslage/u/Satellites/images/Cropped_CCS_{expId}.png"
#image_path = "/home/c/cslage/u/Satellites/images/LSSTCam_Imshow_2025090500272_112.png"
img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)


In [ ]:
[maxima_ridges, minima_ridges, binary_ridges] = find_faint_ridges(img, sigma=12.0)

In [ ]:
threshold_otsu?